In [1]:
import jax
import jax.numpy as jnp
import numpy as np


In [ ]:
alpha= 1.0
beta= 0.4
eta = 0.0

#SGD steps
sgd_steps = 10**1
delta_constant = jnp.maximum(2.0  + ( 2.0 * beta - 1 ) / (alpha ), 2.0 - 1.0 / alpha) + 1.0 #Need to be bigger than 2 + (2 * beta - 1) / (2 * alpha)
print('delta is {:.2f}'.format(delta_constant))

D = 500
V = 5 * D
sgd_batch = 1 #jnp.int32(0.2*D)

omega = 1.0/jnp.float32(D)
traceK = jnp.sum(jnp.arange(1,V+1,dtype=jnp.float32)**(-2*alpha))
print('traceK is {:.2f}'.format(traceK))

key,nkey = jax.random.split(key)
W = jnp.sqrt(omega)*jax.random.normal(nkey, (V,D))
data_scale = jnp.power(jnp.arange(1,V+1,dtype=jnp.float32),-1.0*alpha) #D^(1/2)

#move power-scaling from X's to beta and W to save computation
check_beta = jnp.power(jnp.arange(1,V+1,dtype=jnp.float32),-1.0*(beta+alpha)) #D^(1/2) b
check_beta = jnp.reshape(check_beta,(V,1))

check_W1 = jnp.einsum('i, ij->ij', data_scale, W) #D^(1/2) W
WtranD = jnp.einsum('ij, i->ji', W, data_scale**2) #WtranD
check_W = jnp.reshape(data_scale,(V,1)) * W #D^(1/2) W

hatK = jnp.einsum('ji,jk->ik', check_W1, check_W1)

def ab_oracle(key,batch):
  key, nkey = jax.random.split(key)
  xs = jax.random.normal(nkey, (batch, V))
  A = jnp.tensordot(xs,check_W,1)
  key, nkey = jax.random.split(key)
  noise = jax.random.normal(nkey,(batch, 1))
  y = jnp.tensordot(xs, check_beta,1) + eta*noise
  return A,y

def square_loss(theta):
  v=jnp.tensordot(check_W,theta,1) - check_beta
  return jnp.sum(v*v)